In [82]:
# !pip install  -r ../requirements.txt

In [83]:
import PyPDF2
import re

pdf_path = '../data/documento.pdf'


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                # Añadir todo el texto extraído de la página al texto final
                text += page_text

   # Usar una expresión regular para identificar títulos seguidos por dos puntos (:)
    # Y capturar el texto hasta el próximo título
    pattern = r'([A-Za-zÁÉÍÓÚÑáéíóúñ\s]+:)(.*?)(?=[A-Za-zÁÉÍÓÚÑáéíóúñ\s]+:|$)'
    matches = re.findall(pattern, text, re.DOTALL)

    # Crear una lista que agrupa cada título con su respectivo texto
    paragraphs = []
    for match in matches:
        title = match[0].strip()  # Título
        content = match[1].strip()  # Texto asociado al título
        paragraphs.append(f"{title} {content}")  # Unir título con texto

    return paragraphs


chunks = extract_text_from_pdf(pdf_path)



In [84]:
import cohere
import os
from dotenv import load_dotenv

load_dotenv()
cohere_api_key = "CvPKcfibATr60NC4IDP9wIl0kmoBVDyy449ySK59" # os.getenv["COHERE_API_KEY"]

co = cohere.Client(cohere_api_key)

def generate_embeddings(chunks):
    response = co.embed(texts=chunks, model="large", truncate="RIGHT").embeddings
    return response

# Generar embeddings para los chunks:
embeddings = generate_embeddings(chunks)


In [85]:
import chromadb

# Inicia el cliente de ChromaDB
client = chromadb.Client()

# Crea una colección para almacenar los embeddings
collection = client.create_collection("document_embeddingg")

# Insertar los chunks y embeddings en la colección
def insert_embeddings_in_chromadb(collection, chunks, embeddings):
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        collection.add(wwwzz
            ids = [str(i)], # Added ids argument with unique ID for each chunk
            documents=[chunk],
            embeddings=[embedding],
            metadatas=[{"chunk_id": i}]
        )

# Insertar los embeddings
insert_embeddings_in_chromadb(collection, chunks, embeddings)

UniqueConstraintError: Collection document_embeddingg already exists

In [74]:
def query_embeddings(question):
    # Generar el embedding de la pregunta
    question_embedding = co.embed(texts=[question], model="large", truncate="RIGHT").embeddings[0]

    # Buscar el chunk más similar en ChromaDB
    results = collection.query(query_embeddings=[question_embedding], n_results=1)

    return results["documents"][0]  # Retorna el chunk más relevante


In [75]:
from googletrans import Translator

def translate_text(text, target_language):
    translator = Translator()
    translated = translator.translate(text, dest=target_language)
    return translated.text


In [76]:
from langdetect import detect

def create_prompt_translated(user_question, context):
    # Detectar el idioma de la pregunta
    language = detect(user_question)
    #print(language)
    prompt = f"""
      Contexto: {context}
      Pregunta: {user_question}

      Instruccion:
      - Responder en solo una oración.
      - Siempre responder en el mismo idioma de la pregunta.
      - Responder en tercera persona.
      - Agrega algunos emojis
      """
    #  Agregar emojis dentro del contenido de la respuesta.
    prompt_lang = translate_text(prompt, language)

    return prompt_lang

In [86]:
import cohere

def generate_response(question, context):
    # Generar una respuesta utilizando Cohere
    prompt = create_prompt_translated(question, context)
    print(prompt)

    #model="command-xlarge"
    response = co.generate(prompt=prompt,
                           max_tokens=50,  # Ajusta el número de tokens según sea necesario
                           temperature=0.5 # Ajusta la temperatura para variar la creatividad
                           ).generations[0].text
    return response

# Ejemplo de uso:

question = "Qual è il nome del fiore magico?"
relevant_chunk = query_embeddings(question)
print("----")
response = generate_response(question, relevant_chunk)

print(response)
print(detect(response))

----
Contesto: ['Natura abbagliante: nel profondo della giungla amazzonica, un fiore nmagico noto come "Moonlight" fiorisce solo durante la notte.Con \ npétalos che brilla intensamente, il fiore illumina l'oscurità della giungla, \ nguiando alle creature notturne e rivelando paesaggi abbaglianti.I \ nlugareños credono di avere poteri curativi, rendendolo il tesoro della natura.
Domanda: quale è il noma della magia fiore?

Istruzione:
- Rispondi solo in una preghiera.
- Rispondi sempre nella stessa lingua della domanda.
- Rispondi in terza persona.
- Aggiungi un po 'di emoji


unknown field: parameter model is not a valid field


 🌸 Oh, Magic Flower, you shine bright among the nocturnal creatures of the Amazon Jungle, 🌟 🌟 🌟 🌟 🌟 shining your petals to reveal astounding landscapes that no one would have imagined possible.
en


In [69]:
client.delete_collection("document_embeddingg")